<a href="https://colab.research.google.com/github/AdityaArakere/Sentiment-analysis-model/blob/main/TFModelTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import string
import re
from google.colab import files

uploaded = files.upload()

Saving combined_dataset12.csv to combined_dataset12.csv


In [ ]:
import pandas as pd
import string
import re
from google.colab import files
import io

df = pd.read_csv("combined_dataset12.csv")
df = df[['text', 'sentiment']]

# Preprocess the dataset
# Convert sentiment labels to 0, 1, -1
df['sentiment'] = df['sentiment'].map({1: 4, 0: 0, 2: 2})

# Convert text to lowercase
df['text'] = df['text'].str.lower()

# Remove punctuation
translator = str.maketrans('', '', string.punctuation)
df['text'] = df['text'].astype(str).apply(lambda x: x.translate(translator))

# Remove numbers
df['text'] = df['text'].apply(lambda x: re.sub(r'\d+', '', x))

# Remove extra whitespaces
df['text'] = df['text'].apply(lambda x: ' '.join(x.split()))

# Remove stopwords (optional)
# You can use NLTK or other libraries to remove stopwords if desired

# Print the preprocessed dataset
print(df.head())


                                                text  sentiment
0  i dont know how or why this film has a meager ...          4
1  for a long time it seemed like all the good ca...          4
2  terry gilliams and david peoples teamed up to ...          4
3  what is there to say about an antiestablishmen...          4
4  this movie was made only years after the end o...          4


In [ ]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
!pip install transformers
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import BertTokenizer, TFBertModel
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalMaxPool1D
from tensorflow.keras.callbacks import EarlyStopping

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 93.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 57.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 80.3 MB/s eta 0:00:00


In [ ]:
# Split the dataset into train and test sets
x = df['text'].values
y = df['sentiment'].values

x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=60000, test_size=12000, random_state=42)

# Print the sizes of the train and test sets
print("Train set size:", len(x_train))
print("Test set size:", len(x_test))
print(y_train[0:5])

Train set size: 60000
Test set size: 12000
[2 4 4 2 4]


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Define a custom dataset class
class RedditDataset(tf.keras.utils.Sequence):
    def __init__(self, texts, labels, tokenizer, batch_size):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.batch_size = batch_size

        self.on_epoch_end()

    def __len__(self):
        return len(self.texts) // self.batch_size

    def __getitem__(self, index):
        batch_texts = self.texts[index*self.batch_size:(index+1)*self.batch_size]
        batch_labels = self.labels[index*self.batch_size:(index+1)*self.batch_size]

        batch_encodings = self.tokenizer.batch_encode_plus(
            batch_texts,
            add_special_tokens=True,
            padding='max_length',
            truncation=True,
            max_length=128,
            return_tensors='tf'
        )

        input_tensors = {
            'input_ids': batch_encodings['input_ids'],
            'attention_mask': batch_encodings['attention_mask']
        }

        return input_tensors, batch_labels

    def on_epoch_end(self):
        pass

# Set the hyperparameters
num_epochs = 6
batch_size = 16

tokenizer = BertTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

# Load and preprocess the data
train_dataset = RedditDataset(x_train, y_train, tokenizer, batch_size=batch_size)
test_dataset = RedditDataset(x_test, y_test, tokenizer, batch_size=batch_size)

bertModel = TFBertModel.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

# Get the BERT embeddings
input_ids = tf.keras.layers.Input(shape=(128,), dtype=tf.int32, name='input_ids')
attention_mask = tf.keras.layers.Input(shape=(128,), dtype=tf.int32, name='attention_mask')
embeddings = bertModel.bert(input_ids, attention_mask)[0]

# Apply global max pooling
out = tf.keras.layers.GlobalMaxPooling1D()(embeddings)

# Apply dense layers
out = Dense(64, activation='relu')(out)
out = tf.keras.layers.Dropout(0.3)(out)
out = Dense(32, activation='relu')(out)

# Create the final output layer
y = Dense(5, activation='softmax')(out)

# Create the model with the defined inputs and outputs
model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=y)

# Make the desired layer trainable
model.layers[2].trainable = True

# Define the optimizer
optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=5e-5, decay=0.01, clipnorm=1.0)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)

early_stopping = EarlyStopping(monitor='val_loss', patience=2)

# Compile the model
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

# Train the model
model.fit(train_dataset, epochs=num_epochs, validation_data=test_dataset, callbacks=[early_stopping])

# Evaluate the model
loss, accuracy = model.evaluate(test_dataset)
print(f'Test Loss: {loss:.4f}')
print(f'Test Accuracy: {accuracy * 100:.2f}%')


Some layers from the model checkpoint at nlptown/bert-base-multilingual-uncased-sentiment were not used when initializing TFBertModel: ['dropout_37', 'classifier']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at nlptown/bert-base-multilingual-uncased-sentiment.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Epoch 1/6


3750/3750 [==============================] - 1822s 480ms/step - loss: 0.7949 - accuracy: 0.6759 - val_loss: 0.6767 - val_accuracy: 0.7196
Epoch 2/6
3750/3750 [==============================] - 1797s 479ms/step - loss: 0.6837 - accuracy: 0.7250 - val_loss: 0.6799 - val_accuracy: 0.7253
Epoch 3/6
3750/3750 [==============================] - 1796s 479ms/step - loss: 0.6523 - accuracy: 0.7377 - val_loss: 0.6668 - val_accuracy: 0.7312
Epoch 4/6
3750/3750 [==============================] - 1796s 479ms/step - loss: 0.6381 - accuracy: 0.7451 - val_loss: 0.6635 - val_accuracy: 0.7337
Epoch 5/6
3750/3750 [==============================] - 1792s 478ms/step - loss: 0.6234 - accuracy: 0.7490 - val_loss: 0.6690 - val_accuracy: 0.7337
Epoch 6/6
750/750 [==============================] - 112s 149ms/step - loss: 0.6660 - accuracy: 0.7339
Test Loss: 0.6660
Test Accuracy: 73.39%


In [ ]:
# Define the paths to save the model and tokenizer
model_path = '/content/drive/MyDrive/Models/tf_fine_tuned_model12'
tokenizer_path = '/content/drive/MyDrive/Models/tokenizer12'

# Save the fine-tuned model
model.save(model_path)

# Save the tokenizer
tokenizer.save_pretrained(tokenizer_path)

('/content/drive/MyDrive/Models/tokenizer12/tokenizer_config.json',
 '/content/drive/MyDrive/Models/tokenizer12/special_tokens_map.json',
 '/content/drive/MyDrive/Models/tokenizer12/vocab.txt',
 '/content/drive/MyDrive/Models/tokenizer12/added_tokens.json')

In [ ]:
!pip install transformers
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel


from google.colab import drive
drive.mount('/content/drive')

loaded_model = tf.keras.models.load_model("/content/drive/MyDrive/Models/tf_fine_tuned_model12")

tokenizer = BertTokenizer.from_pretrained('/content/drive/MyDrive/Models/tokenizer12')

test_text = "Louis van Gaal said he had no option but to substitute Paddy McNair in the first half against Southampton because the defender's 'confidence' was shot - but believes that it will benefit the youngster in the long run. The 19-year-old was hooked by Van Gaal after only 39 minutes at St Mary's Stadium on Monday night during Manchester United's 2-1 victory over the Saints. McNair was struggling to contain Southampton strikers Shane Long and Graziano Pelle, forcing Van Gaal into replacing him prematurely. Speaking to Sky Sports after the match, Van Gaal explained: 'He (McNair) hadn't any confidence. He had already given three big chances away. 'I had to (substitute him), it's very disappointing for me and also for Paddy, but I had to because as a manager, I'm responsible to win. 'And I think, after the change, we played a little better.'But in spite of the fact United won the game, McNair was exposed time after time in defence and was substituted - even though Chris Smalling had already departed early with an injury. Jonny Evans came on to replace Smalling, before McNair made way for midfielder Ander Herrera as Michael Carrick dropped back in to the centre of defence in Van Gaal's 3-5-2 system. And, despite admitting it will be difficult for McNair to accept being replaced so early, Van Gaal insisted that it was a necessity which will serve the Northern Irishman well long term. Van Gaal continued: 'Of course, it's tough (for McNair), but it's also in his best interests.' The victory moved United up to third in the Premier League - their highest position since they claimed the title in 2012-13 under Sir Alex Ferguson."

# Tokenize the test input
test_input_encodings = tokenizer.encode_plus(
    test_text,
    add_special_tokens=True,
    padding='max_length',
    truncation=True,
    max_length=128,
    return_tensors='tf'
)

# Get the input tensors
test_input_ids = test_input_encodings['input_ids']
test_attention_mask = test_input_encodings['attention_mask']

# Predict using the model
predictions = loaded_model.predict([test_input_ids, test_attention_mask])

predicted_label = tf.argmax(predictions, axis=1)[0].numpy() + 1

print(predicted_label)

print(predictions)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
1/1 [==============================] - 5s 5s/step
5
[[0.20778748 0.00926053 0.14512989 0.00767588 0.63014627]]
